# Assignment 2

In this assigment, we will work with the *Forest Fire* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/162/forest+fires). Extract the data files into the subdirectory: `../data/fires/` (relative to `./05_src/`).

## Objective

+ The model objective is to predict the area affected by forest fires given the features set. 
+ The objective of this exercise is to assess your ability to construct and evaluate model pipelines.
+ Please note: the instructions are not meant to be 100% prescriptive, but instead they are a set of minimum requirements. If you find predictive performance gains by applying additional steps, by all means show them. 

## Variable Description

From the description file contained in the archive (`forestfires.names`), we obtain the following variable descriptions:

1. X - x-axis spatial coordinate within the Montesinho park map: 1 to 9
2. Y - y-axis spatial coordinate within the Montesinho park map: 2 to 9
3. month - month of the year: "jan" to "dec" 
4. day - day of the week: "mon" to "sun"
5. FFMC - FFMC index from the FWI system: 18.7 to 96.20
6. DMC - DMC index from the FWI system: 1.1 to 291.3 
7. DC - DC index from the FWI system: 7.9 to 860.6 
8. ISI - ISI index from the FWI system: 0.0 to 56.10
9. temp - temperature in Celsius degrees: 2.2 to 33.30
10. RH - relative humidity in %: 15.0 to 100
11. wind - wind speed in km/h: 0.40 to 9.40 
12. rain - outside rain in mm/m2 : 0.0 to 6.4 
13. area - the burned area of the forest (in ha): 0.00 to 1090.84 









### Specific Tasks

+ Construct four model pipelines, out of combinations of the following components:

    + Preprocessors:

        - A simple processor that only scales numeric variables and recodes categorical variables.
        - A transformation preprocessor that scales numeric variables and applies a non-linear transformation.
    
    + Regressor:

        - A baseline regressor, which could be a [K-nearest neighbours model]() or a linear model like [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) or [Ridge Regressors](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ridge_regression.html).
        - An advanced regressor of your choice (e.g., Bagging, Boosting, SVR, etc.). TIP: select a tree-based method such that it does not take too long to run SHAP further below. 

+ Evaluate tune and evaluate each of the four model pipelines. 

    - Select a [performance metric](https://scikit-learn.org/stable/modules/linear_model.html) out of the following options: explained variance, max error, root mean squared error (RMSE), mean absolute error (MAE), r-squared.
    - *TIPS*: 
    
        * Out of the suggested metrics above, [some are correlation metrics, but this is a prediction problem](https://www.tmwr.org/performance#performance). Choose wisely (and don't choose the incorrect options.) 

+ Select the best-performing model and explain its predictions.

    - Provide local explanations.
    - Obtain global explanations and recommend a variable selection strategy.

+ Export your model as a pickle file.


You can work on the Jupyter notebook, as this experiment is fairly short (no need to use sacred). 

# Load the data

Place the files in the ../../05_src/data/fires/ directory and load the appropriate file. 

In [149]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, PowerTransformer, MinMaxScaler, RobustScaler, OneHotEncoder
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingClassifier, StackingRegressor, BaggingRegressor
import pandas as pd
import pickle

In [150]:
# Load data
columns = [
    'coord_x', 'coord_y', 'month', 'day', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind', 'rain', 'area' 
]
fires_dt = (pd.read_csv('../../05_src/data/fires/forestfires.csv', header = 0, names = columns))
fires_dt.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517 entries, 0 to 516
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   coord_x  517 non-null    int64  
 1   coord_y  517 non-null    int64  
 2   month    517 non-null    object 
 3   day      517 non-null    object 
 4   ffmc     517 non-null    float64
 5   dmc      517 non-null    float64
 6   dc       517 non-null    float64
 7   isi      517 non-null    float64
 8   temp     517 non-null    float64
 9   rh       517 non-null    int64  
 10  wind     517 non-null    float64
 11  rain     517 non-null    float64
 12  area     517 non-null    float64
dtypes: float64(8), int64(3), object(2)
memory usage: 52.6+ KB


# Get X and Y

Create the features data frame and target data.

In [151]:
fires_dt_train, fires_dt_test = train_test_split(fires_dt, test_size = 0.7, random_state = 42)

In [152]:
features = fires_dt_train.drop('area', axis=1)
features

,coord_x,coord_y,month,day,ffmc,dmc,dc,isi,temp,rh,wind,rain
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0
256,8,6,aug,sat,94.2,117.2,581.1,11.0,23.9,41,2.2,0.0
462,1,4,sep,sun,91.0,276.3,825.1,7.1,14.5,76,7.6,0.0
100,3,4,aug,sun,91.4,142.4,601.4,10.6,19.8,39,5.4,0.0
226,7,5,oct,mon,91.7,48.5,696.1,11.1,16.1,44,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
71,4,5,sep,fri,94.3,85.1,692.3,15.9,17.7,37,3.6,0.0
106,4,5,mar,thu,91.4,30.7,74.3,7.5,18.2,29,3.1,0.0
270,2,2,aug,tue,92.1,152.6,658.2,14.3,21.8,56,3.1,0.0
435,2,5,jul,sat,90.8,84.7,376.6,5.6,23.8,51,1.8,0.0


In [153]:
target = fires_dt_train['area']
target

4       0.00
256     8.02
462     3.71
100     0.00
226    49.37
       ...  
71      0.00
106     0.00
270     0.52
435     0.00
102     0.00
Name: area, Length: 155, dtype: float64

# Preprocessing

Create two [Column Transformers](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html), called preproc1 and preproc2, with the following guidelines:

- Numerical variables

    * (Preproc 1 and 2) Scaling: use a scaling method of your choice (Standard, Robust, Min-Max). 
    * Preproc 2 only: 
        
        + Choose a transformation for any of your input variables (or several of them). Evaluate if this transformation is convenient.
        + The choice of scaler is up to you.

- Categorical variables: 
    
    * (Preproc 1 and 2) Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) where appropriate.


+ The only difference between preproc1 and preproc2 is the non-linear transformation of the numerical variables.
    


### Preproc 1

Create preproc1 below.

+ Numeric: scaled variables, no other transforms.
+ Categorical: one-hot encoding.

In [154]:
import numpy as np
print(features.drop(['month','day'], axis=1).skew())

coord_x    0.004692
coord_y    0.634378
ffmc      -8.004922
dmc        0.576790
dc        -0.930144
isi        0.458917
temp      -0.296794
rh         1.018368
wind       0.700220
rain       8.880939
dtype: float64


# StandardScaler

In [155]:
transformer1 = ColumnTransformer(
    transformers=[
        ('numeric_transfomer', StandardScaler(), features.columns.drop(['month', 'day'])),
        ('onehotcoding', OneHotEncoder(handle_unknown='ignore'), features.columns.intersection(['month', 'day'])),
    ], remainder='drop')
transformer1

ColumnTransformer(transformers=[('numeric_transfomer', StandardScaler(),
                                 Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind',
       'rain'],
      dtype='object')),
                                ('onehotcoding',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 Index(['month', 'day'], dtype='object'))])

# RobustScaler

In [156]:
transformer2 = ColumnTransformer(
    transformers=[
        ('numeric_transfomer', RobustScaler(), features.columns.drop(['month', 'day'])),
        ('onehotcoding', OneHotEncoder(handle_unknown='ignore'), features.columns.intersection(['month', 'day'])),
    ], remainder='drop')
transformer2

ColumnTransformer(transformers=[('numeric_transfomer', RobustScaler(),
                                 Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind',
       'rain'],
      dtype='object')),
                                ('onehotcoding',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 Index(['month', 'day'], dtype='object'))])

# Min-Max

In [157]:
transformer3 = ColumnTransformer(
    transformers=[
        ('numeric_transfomer', MinMaxScaler(), features.columns.drop(['month', 'day'])),
        ('onehotcoding', OneHotEncoder(handle_unknown='ignore'), features.columns.intersection(['month', 'day'])),
    ], remainder='drop')
transformer3

ColumnTransformer(transformers=[('numeric_transfomer', MinMaxScaler(),
                                 Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind',
       'rain'],
      dtype='object')),
                                ('onehotcoding',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 Index(['month', 'day'], dtype='object'))])

### Preproc 2

Create preproc2 below.

+ Numeric: scaled variables, non-linear transformation to one or more variables.
+ Categorical: one-hot encoding.

In [158]:
transformer = ColumnTransformer(
    transformers=[
        ('numeric_transfomer', StandardScaler(), features.columns.drop(['month', 'day'])),
        ('power_transform',  PowerTransformer(), features.columns.intersection(['ffmc','rain'])),
        ('onehotcoding', OneHotEncoder(handle_unknown='ignore'), features.columns.intersection(['month', 'day'])),
    ], remainder='drop')
transformer

ColumnTransformer(transformers=[('numeric_transfomer', StandardScaler(),
                                 Index(['coord_x', 'coord_y', 'ffmc', 'dmc', 'dc', 'isi', 'temp', 'rh', 'wind',
       'rain'],
      dtype='object')),
                                ('power_transform', PowerTransformer(),
                                 Index(['ffmc', 'rain'], dtype='object')),
                                ('onehotcoding',
                                 OneHotEncoder(handle_unknown='ignore'),
                                 Index(['month', 'day'], dtype='object'))])

## Model Pipeline


Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `regressor` and assign a regression model to it. 

## Regressor

+ Use a regression model to perform a prediction. 

    - Choose a baseline regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Choose a more advance regressor, tune it (if necessary) using grid search, and evaluate it using cross-validation.
    - Both model choices are up to you, feel free to experiment.

In [159]:
# Pipeline A = preproc1 + baseline
pipeline_A = Pipeline(
    [
        ('preprocessing', transformer1),
        ('regressor', Ridge())
    ]
)

In [160]:
# Pipeline B = preproc2 + baseline
pipeline_B = Pipeline(
    [
        ('preprocessing', transformer),
        ('regressor', Ridge())
    ]
)

In [161]:
# Pipeline C = preproc1 + advanced model
pipeline_C = Pipeline(
    [
        ('preprocessing', transformer1),
        ('regressor', RandomForestRegressor())
    ]
)

In [162]:
# Pipeline D = preproc2 + advanced model
pipeline_D = Pipeline(
    [
        ('preprocessing', transformer),
        ('regressor', RandomForestRegressor())
    ]
)

# Tune Hyperparams

+ Perform GridSearch on each of the four pipelines. 
+ Tune at least one hyperparameter per pipeline.
+ Experiment with at least four value combinations per pipeline.

In [163]:
param_grid = {
    'regressor__alpha': [0.0001, 0.1, 10, 20, 30]
}

scoring=['neg_root_mean_squared_error', 'r2']

grid_cv_A = GridSearchCV(
    estimator = pipeline_A,
    param_grid = param_grid,
    scoring = scoring,
    cv = 5,
    refit = 'neg_root_mean_squared_error'
)

# Fitting a model on training data
grid_cv_A.fit(features, target)

results = pd.DataFrame(grid_cv_A.cv_results_)
results[['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_regressor__alpha', 'params', 'mean_test_neg_root_mean_squared_error', 'std_test_neg_root_mean_squared_error',
       'rank_test_neg_root_mean_squared_error', 'mean_test_r2', 'std_test_r2', 'rank_test_r2']].sort_values('rank_test_neg_root_mean_squared_error')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__alpha,params,mean_test_neg_root_mean_squared_error,std_test_neg_root_mean_squared_error,rank_test_neg_root_mean_squared_error,mean_test_r2,std_test_r2,rank_test_r2
4,0.011003,0.001262,0.006993,0.000640,30.0000,{'regressor__alpha': 30},-47.917897,44.440454,1,-1.265548,1.782796,1
3,0.010801,0.000985,0.007199,0.000749,20.0000,{'regressor__alpha': 20},-48.795783,43.973256,2,-1.499483,2.058250,2
2,0.012598,0.001624,0.006798,0.001325,10.0000,{'regressor__alpha': 10},-50.441598,43.135868,3,-1.954021,2.577984,3
1,0.013197,0.001165,0.007603,0.000487,0.1000,{'regressor__alpha': 0.1},-56.047278,40.251592,4,-4.269027,5.829471,4
0,0.011601,0.001023,0.007573,0.000983,0.0001,{'regressor__alpha': 0.0001},-56.647559,39.953659,5,-4.608685,6.366254,5


In [164]:
# Best parameter and score
print("Pipeline A - Best Parameter:", grid_cv_A.best_params_)
print("Pipeline A - Best RMSE:", -grid_cv_A.best_score_)

Pipeline A - Best Parameter: {'regressor__alpha': 30}
Pipeline A - Best RMSE: 47.917897082181824


In [165]:
param_grid = {
    'regressor__alpha': [0.01, 0.1, 10, 20, 30]
}

grid_cv_B = GridSearchCV(
    estimator = pipeline_B,
    param_grid = param_grid,
    scoring = scoring,
    cv = 5,
    refit = 'neg_root_mean_squared_error'
)

# Fitting a model on training data
grid_cv_B.fit(features, target)

results = pd.DataFrame(grid_cv_B.cv_results_)
results[['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_regressor__alpha', 'params', 'mean_test_neg_root_mean_squared_error', 'std_test_neg_root_mean_squared_error',
       'rank_test_neg_root_mean_squared_error', 'mean_test_r2', 'std_test_r2', 'rank_test_r2']].sort_values('rank_test_neg_root_mean_squared_error')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__alpha,params,mean_test_neg_root_mean_squared_error,std_test_neg_root_mean_squared_error,rank_test_neg_root_mean_squared_error,mean_test_r2,std_test_r2,rank_test_r2
4,0.019598,0.001019,0.008401,0.000801,30.00,{'regressor__alpha': 30},-48.076583,44.376696,1,-1.265742,1.739299,1
3,0.020199,0.001326,0.008800,0.000748,20.00,{'regressor__alpha': 20},-48.962778,43.911190,2,-1.497608,2.009740,2
2,0.019599,0.001018,0.008601,0.000489,10.00,{'regressor__alpha': 10},-50.622476,43.070420,3,-1.952038,2.527758,3
1,0.019203,0.001727,0.009597,0.001017,0.10,{'regressor__alpha': 0.1},-56.484110,40.053287,4,-4.409331,6.006063,4
0,0.028398,0.008594,0.010399,0.001852,0.01,{'regressor__alpha': 0.01},-57.357764,39.616841,5,-4.786119,6.432441,5


In [166]:
# Best parameter and score
print("Pipeline B - Best Parameters:", grid_cv_B.best_params_)
print("Pipeline B - Best RMSE:", -grid_cv_B.best_score_)

Pipeline B - Best Parameters: {'regressor__alpha': 30}
Pipeline B - Best RMSE: 48.07658289170647


In [167]:
param_grid = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__max_depth': [None, 10, 20]
}

grid_cv_C = GridSearchCV(
    estimator = pipeline_C,
    param_grid = param_grid,
    scoring = scoring,
    cv = 5,
    refit = 'neg_root_mean_squared_error'
)

# Fitting a model on training data
grid_cv_C.fit(features, target)

results = pd.DataFrame(grid_cv_C.cv_results_)
results[['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_regressor__max_depth', 'param_regressor__n_estimators', 'params', 'mean_test_neg_root_mean_squared_error', 'std_test_neg_root_mean_squared_error',
       'rank_test_neg_root_mean_squared_error', 'mean_test_r2', 'std_test_r2', 'rank_test_r2']].sort_values('rank_test_neg_root_mean_squared_error')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__max_depth,param_regressor__n_estimators,params,mean_test_neg_root_mean_squared_error,std_test_neg_root_mean_squared_error,rank_test_neg_root_mean_squared_error,mean_test_r2,std_test_r2,rank_test_r2
1,0.209862,0.016789,0.013200,0.001164,None,100,"{'regressor__max_depth': None, 'regressor__n_e...",-54.516211,40.601352,1,-4.623046,7.242203,2
6,0.109004,0.011624,0.010330,0.000850,20,50,"{'regressor__max_depth': 20, 'regressor__n_est...",-55.066760,40.029718,2,-5.370496,8.884329,4
3,0.099996,0.002280,0.009202,0.000981,10,50,"{'regressor__max_depth': 10, 'regressor__n_est...",-55.361562,41.138095,3,-5.004892,8.243413,3
5,0.370207,0.006829,0.017995,0.001089,10,200,"{'regressor__max_depth': 10, 'regressor__n_est...",-55.454978,40.996116,4,-5.602089,9.404859,7
0,0.114407,0.014816,0.009397,0.000799,None,50,"{'regressor__max_depth': None, 'regressor__n_e...",-55.508182,39.857474,5,-4.577818,6.810954,1
2,0.418666,0.047493,0.018127,0.001124,None,200,"{'regressor__max_depth': None, 'regressor__n_e...",-55.649485,40.796263,6,-5.472224,8.792042,6
4,0.198305,0.013686,0.011387,0.000804,10,100,"{'regressor__max_depth': 10, 'regressor__n_est...",-55.779771,39.882980,7,-5.419567,8.613041,5
8,0.384685,0.019163,0.018401,0.001021,20,200,"{'regressor__max_depth': 20, 'regressor__n_est...",-55.823492,40.678002,8,-5.731390,9.328208,8
7,0.194601,0.003007,0.012204,0.000407,20,100,"{'regressor__max_depth': 20, 'regressor__n_est...",-57.983698,41.603493,9,-9.088625,16.688132,9


In [168]:
# Best parameter and score
print("Pipeline C - Best Parameters:", grid_cv_C.best_params_)
print("Pipeline C - Best RMSE:", -grid_cv_C.best_score_)

Pipeline C - Best Parameters: {'regressor__max_depth': None, 'regressor__n_estimators': 100}
Pipeline C - Best RMSE: 54.516211243058066


In [169]:
param_grid = {
    'regressor__n_estimators': [50, 100, 200],
    'regressor__max_depth': [None, 10, 20]
}

grid_cv_D = GridSearchCV(
    estimator = pipeline_D,
    param_grid = param_grid,
    scoring = scoring,
    cv = 5,
    refit = 'neg_root_mean_squared_error'
)

# Fitting a model on training data
grid_cv_D.fit(features, target)

results = pd.DataFrame(grid_cv_D.cv_results_)
results[['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_regressor__max_depth', 'param_regressor__n_estimators', 'params', 'mean_test_neg_root_mean_squared_error', 'std_test_neg_root_mean_squared_error',
       'rank_test_neg_root_mean_squared_error', 'mean_test_r2', 'std_test_r2', 'rank_test_r2']].sort_values('rank_test_neg_root_mean_squared_error')

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_regressor__max_depth,param_regressor__n_estimators,params,mean_test_neg_root_mean_squared_error,std_test_neg_root_mean_squared_error,rank_test_neg_root_mean_squared_error,mean_test_r2,std_test_r2,rank_test_r2
6,0.115414,0.009485,0.010801,0.000750,20,50,"{'regressor__max_depth': 20, 'regressor__n_est...",-53.937302,40.802485,1,-3.539405,5.063928,1
5,0.391407,0.013417,0.021395,0.001955,10,200,"{'regressor__max_depth': 10, 'regressor__n_est...",-54.068052,40.771074,2,-4.759073,7.783620,2
8,0.397274,0.015564,0.018799,0.000750,20,200,"{'regressor__max_depth': 20, 'regressor__n_est...",-54.558066,41.124452,3,-4.929559,8.206596,4
2,0.406711,0.008898,0.020200,0.002041,None,200,"{'regressor__max_depth': None, 'regressor__n_e...",-54.775290,40.051350,4,-4.768261,7.470568,3
0,0.118589,0.007083,0.010599,0.000493,None,50,"{'regressor__max_depth': None, 'regressor__n_e...",-54.939132,40.686637,5,-5.135848,8.421161,6
1,0.215698,0.016945,0.013802,0.000401,None,100,"{'regressor__max_depth': None, 'regressor__n_e...",-55.491276,40.281589,6,-5.780613,9.756884,7
4,0.208735,0.010856,0.014598,0.001745,10,100,"{'regressor__max_depth': 10, 'regressor__n_est...",-55.937194,40.258430,7,-4.930497,7.240243,5
7,0.206404,0.002058,0.013996,0.000632,20,100,"{'regressor__max_depth': 20, 'regressor__n_est...",-57.709694,40.474832,8,-7.670384,13.425305,9
3,0.109801,0.003921,0.011602,0.001498,10,50,"{'regressor__max_depth': 10, 'regressor__n_est...",-59.117876,38.382256,9,-6.524506,8.637149,8


In [170]:
# Best parameter and score
print("Pipeline D - Best Parameters:", grid_cv_D.best_params_)
print("Pipeline D - Best RMSE:", -grid_cv_D.best_score_)

Pipeline D - Best Parameters: {'regressor__max_depth': 20, 'regressor__n_estimators': 50}
Pipeline D - Best RMSE: 53.93730224657084


# Evaluate

+ Which model has the best performance?  
Preproc 1 + Ridge is doing the best with RMSE is 47.92.

# Export

+ Save the best performing model to a pickle file.

In [171]:
with open('pipeline_A.pkl', 'wb') as file_A:
    pickle.dump(grid_cv_A.best_estimator_, file_A)

# Explain

+ Use SHAP values to explain the following only for the best-performing model:

    - Select an observation in your test set and explain which are the most important features that explain that observation's specific prediction.

    - In general, across the complete training set, which features are the most and least important.

+ If you were to remove features from the model, which ones would you remove? Why? How would you test that these features are actually enhancing model performance?

*(Answer here.)*

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at the `help` channel. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Cortez,Paulo and Morais,Anbal. (2008). Forest Fires. UCI Machine Learning Repository. https://doi.org/10.24432/C5D88D.